# Divergent data reduction for Amor

In this notebook, we will look at how to use the `essreflectometry` package with Sciline, for reflectometry data collected from the PSI instrument [Amor](https://www.psi.ch/en/sinq/amor) in [divergent beam mode](https://www.psi.ch/en/sinq/amor/selene).

We will begin by importing the modules that are necessary for this notebook.

In [ ]:
import scipp as sc
import sciline
from essreflectometry.amor import providers, default_parameters
from essreflectometry.types import *

In [ ]:
params = {
    **default_parameters,
    QBins: sc.geomspace(dim='Q', start=0.008, stop=0.075, num=200, unit='1/angstrom'),
    SampleRotation[Sample]: sc.scalar(0.7989, unit='deg'),
    Filename[Sample]: "sample.nxs",
    SampleRotation[Reference]: sc.scalar(0.8389, unit='deg'),
    Filename[Reference]: "reference.nxs",
    WavelengthEdges: sc.array(dims=['wavelength'], values=[2.4, 16.0], unit='angstrom'),
}

In [ ]:
pipeline = sciline.Pipeline(
    providers,
    params=params
)

In [ ]:
pipeline.visualize((NormalizedIofQ, QResolution), graph_attr={'rankdir': 'LR'})

In [ ]:
# Compute I over Q and the standard deviation of Q
ioq, qstd = pipeline.compute((NormalizedIofQ, QResolution)).values()

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(5, 7))
ax1 = fig.add_axes([0, 0.55, 1.0, 0.45])
ax2 = fig.add_axes([0, 0.0, 1.0, 0.45])
cax = fig.add_axes([1.05, 0.55, 0.03, 0.45])
fig1 = ioq.plot(norm='log', ax=ax1, cax=cax, grid=True)
fig2 = ioq.mean('detector_number').plot(norm='log', ax=ax2, grid=True)
fig1.canvas.xrange = fig2.canvas.xrange

## Make a $(\lambda, \theta)$ map
A good sanity check is to create a two-dimensional map of the counts in $\lambda$ and $\theta$ bins. To achieve this, we request the `ThetaData` from the pipeline. In the graph above we can see that `WavelengthData` is required to compute `ThetaData`, therefore it is also present in `ThetaData` so we don't need to require it separately.

In [ ]:
from essreflectometry.types import ThetaData
pipeline.compute(ThetaData[Sample])\
    .bins.concat('detector_number')\
    .hist(
        theta=sc.linspace(dim='theta', start=0.0, stop=1.2, num=165, unit='deg').to(unit='rad'),
        wavelength=sc.linspace(dim='wavelength', start=0, stop=15.0, num=165, unit='angstrom'),
    )\
    .plot()


This plot can be used to check if the value of the sample rotation angle $\omega$ is correct. The bright triangles should be pointing back to the origin $\lambda = \theta = 0$.

## Save data

We can save the computed $I(Q)$ to an [ORSO](https://www.reflectometry.org) [.ort](https://github.com/reflectivity/file_format/blob/master/specification.md) file using the [orsopy](https://orsopy.readthedocs.io/en/latest/index.html) package.

First, we need to collect the metadata for that file.
To this end, we build a pipeline with additional providers.
We also insert a parameter to indicate the creator of the processed data.

In [ ]:
from essreflectometry import orso
from essreflectometry.amor import orso as amor_orso
from orsopy import fileio

In [ ]:
providers_with_metadata = (
    *providers,
    *orso.providers,
    *amor_orso.providers,
)

params[orso.OrsoCreator] = orso.OrsoCreator(fileio.base.Person(
    name='Max Mustermann',
    affiliation='European Spallation Source ERIC',
    contact='max.mustermann@ess.eu',
))

metadata_pipeline = sciline.Pipeline(
    providers_with_metadata,
    params=params
)

Then, we recompute $I(Q)$ and and combine it with the ORSO metadata:

In [ ]:
iofq_dataset = metadata_pipeline.compute(orso.OrsoIofQDataset)

Unfortunately, some metadata could not be determined automatically.
In particular, we need to specify the sample manually:

In [ ]:
iofq_dataset.info.data_source.sample

In [ ]:
iofq_dataset.info.data_source.sample = fileio.data_source.Sample(
    name='Ni / Ti Multilayer',
    model=fileio.data_source.SampleModel(
      stack='air | (Ni | Ti) * 5 | Si',
    ),
)

And we also add the URL of this notebook to make it easier to reproduce the data:

In [ ]:
iofq_dataset.info.reduction.script = 'https://scipp.github.io/essreflectometry/examples/amor.html'

To support tracking provenance, we also list the corrections that were done by the workflow and store them in the dataset:

In [ ]:
iofq_dataset.info.reduction.corrections = orso.find_corrections(metadata_pipeline.get(orso.OrsoIofQDataset))

Finally, we can save the data to a file.
Note that `iofq_dataset` is an [orsopy.fileio.orso.OrsoDataset](https://orsopy.readthedocs.io/en/latest/orsopy.fileio.orso.html#orsopy.fileio.orso.OrsoDataset).

In [ ]:
iofq_dataset.save('amor_reduced_iofq.ort')

Look at the first 50 lines of the file to inspect the metadata:

In [ ]:
!head amor_reduced_iofq.ort -n50